In [ ]:
import os
os.chdir("../")


In [ ]:
from scripts.load_data import load_df
df = load_df()

In [ ]:
print(df.columns)

In [ ]:
from scripts.preprocessing import count_missing_values, replace_missing_with_mean, replce_messing_with_median, count_outliers, replace_outliers_with_quartile

mv = count_missing_values(df)
columns = ['Bearer Id', 'Start ms', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1'
       , 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',
       'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B',
       'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
       'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
       'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
       'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
       'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)',
       'Total UL (Bytes)', 'Total DL (Bytes)']


df=replace_outliers_with_quartile(df,columns)

df = replace_missing_with_mean(df, "TCP DL Retrans. Vol (Bytes)")



In [ ]:

import pandas as pd
from scripts.eda_module import (
    load_and_inspect_data,
    handle_missing_and_outliers,
    segment_users_by_duration,
    compute_basic_metrics,
    univariate_analysis,
    bivariate_analysis,
    correlation_analysis,
    pca_analysis,
    aggregate_user_data  
)

# Call the aggregation function
user_aggregates = aggregate_user_data(df)

# Preview the result
print(user_aggregates.head())


In [ ]:
# main_script.py
import pandas as pd
from user_engagement import (
    aggregate_engagement_metrics,
    normalize_and_cluster,
    compute_cluster_stats,
    aggregate_traffic_per_app,
    plot_top_3_apps,
    elbow_method,
    silhouette_analysis
)

# Load your dataset
df = pd.read_csv('your_dataset.csv')

# Task 1: Aggregate engagement metrics
user_metrics = aggregate_engagement_metrics(df)
print(user_metrics.head())

# Task 2: Normalize and apply k-means clustering
user_metrics, kmeans = normalize_and_cluster(user_metrics, k=3)
print(user_metrics.head())

# Task 3: Compute cluster stats
cluster_stats = compute_cluster_stats(user_metrics)
print(cluster_stats)

# Task 4: Aggregate traffic per application
top_10_users_per_app = aggregate_traffic_per_app(df)
print(top_10_users_per_app)

# Task 5: Plot top 3 most used applications
plot_top_3_apps(top_10_users_per_app)

# Task 6: Use elbow method to find optimal k
inertia_values = elbow_method(user_metrics)

# Task 7: Evaluate silhouette score for k=3
silhouette_score_k3 = silhouette_analysis(user_metrics, k=3)
print(f"Silhouette Score for k=3: {silhouette_score_k3}")
